In [ ]:
from heapq import heappush, heappop
class Noeud:
    def __init__(self,
                 valeur: int,
                 lettre: str = "",
                 filsG = None,
                 filsD = None) -> None:
        self.valeur = valeur
        self.lettre = lettre
        self.filsG = filsG
        self.filsD = filsD

    def __lt__(self, n) -> bool:
        """ 
        Teste si la valeur du noeud est bien inférieure à la valeur du noeud n.  
        """
        return self.valeur < n.valeur
def est_vide(n) -> bool:
    """ Vérifie que n est le noeud vide """
    return n is None
def est_feuille(n) -> bool:
    """ Vérifie que le noeud est une feuille. """
    return est_vide(n.filsD) and est_vide(n.filsG)
def parcours(n) -> None:
    """Parcours préfixe de l'arbre """
    if not est_vide(n):
        print(n.valeur, end=" ")
        parcours(n.filsG)
        parcours(n.filsD)
    else:
        print(".", end=" ")
def creation_dict_frequences(message: str):
    """
    Établit la table des fréquences des caractères dans message.
    """
    d = {}
    for car in message:
        if car not in d.keys():
            d[car] = 1
        else:
            d[car] = d[car] + 1
    return d
def construction_arbre_huffman(dic_frequences):
    """
    Construction de l'arbre de Huffman.
    """
    file = []

    # Intégration de tous les noeuds dans la file
    for cle in dic_frequences.keys():
        z = Noeud(dic_frequences[cle], cle)
        heappush(file, z)

    # Construction de l'Arbre
    for i in range(len(dic_frequences) - 1):
        x = heappop(file)
        y = heappop(file)

        z = Noeud(x.valeur + y.valeur)
        z.filsG = y
        z.filsD = x

        heappush(file, z)

    return heappop(file)
def codes_huffman_parcours(n, dic, code: str) -> None:
    """
    Parcours de l'arbre et construction des codes et du dictionnaire passé en argument. Les lettres constituent les valeurs et les codes les clés.

    Algorithme récursif.
    """
    if est_feuille(n):
        dic[n.lettre] = code
    else:
        codes_huffman_parcours(n.filsG, dic, code + "0")
        codes_huffman_parcours(n.filsD, dic, code + "1")
def encodage(message: str, codes) -> str:
    """
    Retourne la chaîne de bits produite par le codage de Huffman pour la chaîne message.
    """
    rep = ""
    for car in message:
        rep += codes[car]
    return rep
def decodage(message_compresse: str, codes) -> str:
    """
    Retourne le message non compressé à partir du codage de Huffman. 
    """
    rep = ""
    temp = ""
    for code in message_compresse:
        temp += code
        for (lettre, code_lettre) in codes.items():
            if temp == code_lettre:
                rep += lettre
                temp = ""
    return rep
def lire_fichier(texte) :
    ligne=[]
    with open(texte, 'r') as txt:
        texte=""
        lignes = txt.readlines()
        for line in lignes:
            ligne.append(line)
        for i in ligne:
            texte=texte+i+""
    return texte 
if __name__ == "__main__":
    assert creation_dict_frequences("ABRACADABRA") == {'A': 5,'B': 2,'R': 2,'C': 1,'D': 1}

    message = lire_fichier('txt.txt')
    frequences = {'A': 300,'B': 350,'C': 160,'D': 510,'E': 280,'F': 490,'G': 450,'H': 350}
    
    print("Lecture du fichier")
    print(lire_fichier('txt.txt'))
    texte_a_encoder=lire_fichier('txt.txt')
    
    print("Dictionnaire des fréquences :")
    print(frequences)

    print("\nParcours de l'arbre :")
    arbre = construction_arbre_huffman(frequences)
    parcours(arbre)

    dic = {}  # Dictionnaire des codes
    codes_huffman_parcours(arbre, dic, "")
    print("\n\nCodes : ")
    print(dic)

    message_code = encodage("CACH", dic)
    print("\nEncodage de CACHE")
    print(message_code)

    message_decode = decodage(message_code, dic)
    print("\nDécodage de CACHE")
    print(message_decode)

: 

In [27]:
import bisect
import json
class ArbreHuffman():
    def __init__(self,lettre,nbocc,g=None,d=None):
        self.lettre=lettre
        self.nbocc=nbocc
        self.gauche=g
        self.droite=d
    def est_feuille(self)->bool:
        return not(self.gauche) and not(self.droite)
    def __lt__(self,other):
        return self.nbocc>other.nbocc
    
def parcours(arbre, chemin_en_cours,dico):
    if not arbre:
        return
    if arbre.est_feuille():
        dico[arbre.lettre]=chemin_en_cours
    else:
        parcours(arbre.gauche, chemin_en_cours + [0], dico)
        parcours(arbre.droite, chemin_en_cours + [1], dico)
        
def fusionne(gauche,droite):
    nbocc_total=gauche.nbocc+droite.nbocc
    return ArbreHuffman(None,nbocc_total,gauche,droite)

def compte_occurences(texte:str)->dict:
    occ=dict()
    for car in texte:
        if car not in occ:
            occ[f"{car}"]=0
        occ[f"{car}"]=occ[f"{car}"]+1
    return occ

def construit_liste_arbres(texte:str)->list:
    dic_occurences=compte_occurences(texte)
    liste_arbres=[]
    for lettre,occ in dic_occurences.items():
        liste_arbres.append(ArbreHuffman(lettre,occ))
    return liste_arbres

def codage_huffman(texte:str)->dict:
    liste_arbres=construit_liste_arbres(texte)
    liste_arbres.sort()
    while len(liste_arbres)>1:
        droite=liste_arbres.pop()
        gauche=liste_arbres.pop()
        new_arbre=fusionne(gauche, droite)
        bisect.insort(liste_arbres, new_arbre)
    arbre_huffman=liste_arbres.pop()
    dico={}
    parcours(arbre_huffman,[],dico)
    encode_message(dico,texte)
    return dico

def lire_fichier(texte) :
    with open(texte, 'r') as txt:
        lignes = txt.read()
        return lignes

def encode_message(dico,texte)->str:
    encoded=""
    for car in texte:
        for val in dico[car]:
            encoded=encoded+f"{val}"
    sauvegarde(encoded,dico)
    return encoded

def sauvegarde(encoded,dico):
    file=open("huffman.txt",mode="w")
    file.write(encoded)
    file.close()
    dic=open("decodage.txt",mode="w")
    dic.write(json.dumps(dico))
    dic.close()

def decrypte(textfile, codefile):
    decrypted,current=[],""
    with open(textfile) as txt:
        texte=txt.read()
    with open(codefile) as dic:
        dictionnaire = dic.read()
    dico = dictionnaire.replace("'", '"') 
    dico = json.loads(dico,encoding='utf-8')
    reverse_code = dict(zip(["".join([str(y) for y in v]) for v in dico.values()], dico.keys()))
    for c in texte:
        current += c
        if current in reverse_code:
            decrypted.append(reverse_code[current])
            current = ""
    return "".join(decrypted)



if __name__=='__main__':
    texte=lire_fichier("txt.txt")
    dico=codage_huffman(texte)
    #print(dico)
    print(compte_occurences(texte))
    print(decrypte("huffman.txt","decodage.txt"))
    #print(compte_occurences(texte))

{'J': 2, 'a': 44, 's': 36, 'o': 34, 'n': 52, ' ': 137, 'é': 10, 't': 43, 'i': 32, 'u': 31, 'e': 74, 'f': 5, '.': 6, 'I': 3, 'l': 33, 'd': 26, ',': 11, 'p': 11, '1': 1, '5': 1, 'à': 4, 'U': 8, 'B': 1, 'O': 1, 'h': 5, 'y': 6, 'q': 7, 'r': 34, 'g': 6, 'è': 7, 'b': 5, 'm': 7, ':': 2, '"': 6, 'E': 4, '-': 1, 'c': 12, '?': 1, 'û': 1, 'L': 7, 'C': 3, 'x': 2, 'R': 3, 'S': 2, 'T': 3, 'H': 3, 'A': 2, 'G': 2, 'M': 2, 'V': 1, 'j': 2, 'D': 1, 'v': 1, 'Q': 1, 'F': 2, '7': 1, '4': 1, '8': 2, '0': 1, '3': 1, '!': 1}
Jason était un enfant. Il étudia, pendant 15 ans, à l UBO, la physique. Il rédigea une thèse répondant à la problématique :  "Est-ce que l informatique est de la science ?". Il dût s appuyer, sous contrainte, sur le témoignage du doyen du lycée La Croix Rouge La Salle : "Tony HAUGUEL". Ce dernier l insulta très fort en prenant compte de sa nomination. La légende raconte que Monsieur Jourdain a entendu des insultes telles que "Va bien te faire enculer sale petit arabe juif roux". Dès lors, 

https://sigscrb-my.sharepoint.com/:i:/g/personal/perrotl_lacroixrouge-brest_fr/EZVlzbAuZZxPsaq_7-Q7tW0B3QVZCeQ-lO84pWeX6PF2ag

<div class="test">
<img src="Images/projet.png" alt="Huffman" width=100% align=center>
<div>